In [8]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd

In [9]:
# подробнее о каждой заявке по ссылке
def get_more_info(link):
    url = windowOrigin + link
    r = requests.get(url)
    return r.text

# функция для замены следующие элементы \r \n \t 
def get_text_without_tabs(text):
    return text.replace('\r', '').replace('\n','').replace('\t', '')

# функция для парсинга html страницы по тегам и добавления данных в DataFrame
def parse_data_to_df(response):
    text = response

    soup = BeautifulSoup(text)
    content = soup.find('div', {'class': 'content'})
    items_list = content.find('div', {'class': 'container'}).find('div', {'class': 'container', 'style': ''}).find('div', {'class': 'row'})
    items = items_list.find_all('div', {'class': 'col-sm-12 col-md-12 col-lg-12'})
    
    for item in items:        
        request_link = item.find('h5', {'class': 'card-title'}).find('a').get('href')
        details_html = BeautifulSoup(get_more_info(request_link))
        details = details_html.find('div', {'class': 'content'}).find('div', {'class': 'container'}).find('div', {'class': 'container'})
        title = get_text_without_tabs(details.find('h4', {'class': 'card-title'}).text)
        rows = details.find('table').find_all('tr')
        
        organizer = get_text_without_tabs(details.find('table').find_all('tr')[0].find_all("td")[1].text)
        start_date = get_text_without_tabs(details.find('table').find_all('tr')[1].find_all("td")[1].text)
        end_date = get_text_without_tabs(details.find('table').find_all('tr')[2].find_all("td")[1].text)
        status = get_text_without_tabs(details.find('table').find_all('tr')[3].find_all("td")[1].text)
        
#         print("org: " + organizer)
#         print("start_date: " + start_date)
#         print("end_date: " + end_date)
#         print("status: " + status)
        allData.loc[len(allData)]=[title, start_date, end_date, status, organizer]

In [10]:
windowOrigin = "https://zakup.kbtu.kz"

allData = pd.DataFrame(columns=["Название","Дата начала","Дата окончания","Статус","Организатор"])
allData

pageCount = 47 # в общем количество страниц
for i in range(1, pageCount):
    url = 'https://zakup.kbtu.kz/zakupki/sposobom-zaprosa-cenovyh-predlozheniy&page=' + str(i) # данные с каждой страницы
    response = requests.get(url).text
    parse_data_to_df(response) # фунция для парсинга и записи в DataFrame

In [11]:
allData

,Название,Дата начала,Дата окончания,Статус,Организатор
0,"Закуп товара для ФЭиНИ, согласно заявке № 244...",06.09.2021 12:45,09.09.2021 12:45,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
1,"Закуп санитарно-противоэпидемиологических, де...",03.09.2021 14:00,08.09.2021 14:00,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
2,"Закуп товара для НОЦАЭиН, АО ""КБТУ"", согласно...",02.09.2021 17:00,07.09.2021 17:00,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
3,"Закуп хоз.товаров и медикаментов для ДМиС, со...",01.09.2021 15:10,06.09.2021 15:11,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
4,Закуп услуг по изготовлению печати КБТУ _ ДБУ...,01.09.2021 12:45,06.09.2021 12:45,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
...,...,...,...,...,...
225,Крестовина из нержавеющей стали по чертежам,03.09.2020 15:00,08.09.2020 15:01,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
226,Закуп оргстекла,03.09.2020 15:00,08.09.2020 15:01,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
227,"Закупка товаров для ИЦ, согласно заявке № 100...",02.09.2020 10:00,07.09.2020 10:01,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...
228,"Закупка товаров для ДИТ, согласно заявке № 96...",28.08.2020 15:00,03.09.2020 15:01,ЗАВЕРШЕНО,АО Казахстанско-Британский технический универс...


In [12]:
allData.to_csv('testTask.csv',encoding='utf-8-sig')